[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/keras-team/autokeras/blob/master/docs/templates/tutorial/image_classification.ipynb)

In [1]:
import tensorflow as tf
import autokeras as ak
tf.__version__, ak.__version__

('2.3.1', '1.0.11')

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

## Load MNIST dataset

In [3]:
from tensorflow.keras.datasets import cifar10, fashion_mnist, mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('Training image shape:', x_train.shape) # (60000, 28, 28)
print('Training label shape:', y_train.shape) # (60000,)
print('First five training labels:', y_train[:5]) # array([5 0 4 1 9], dtype=uint8)

Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


## ResNetBlock

In [4]:
import timeit

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ResNetBlock()(output_node) # version="v2", pretrained=False
output_node = ak.ClassificationHead()(output_node)

resnet_auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=3,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
resnet_auto_model.fit(x_train, y_train, epochs=10, batch_size=64)
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

Trial 1 Complete [00h 33m 19s]
val_loss: 0.04570947587490082

Best val_loss So Far: 0.04570947587490082
Total elapsed time: 00h 33m 19s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
res_net_block_1...|False             |False             
res_net_block_1...|resnet50          |resnet50          
res_net_block_1...|True              |True              
classification_...|flatten           |flatten           
classification_...|0                 |0                 
optimizer         |adam              |adam              
learning_rate     |0.1               |0.001             

Epoch 1/10
541/751 [====================>.........] - ETA: 51s - loss: 55.0168 - accuracy: 0.5566

KeyboardInterrupt: 

### Get the summarized results during the tuning process

In [5]:
resnet_auto_model.tuner.results_summary()

Results summary
Results in ./auto_model
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
res_net_block_1/pretrained: False
res_net_block_1/version: resnet50
res_net_block_1/imagenet_size: True
classification_head_1/spatial_reduction_1/reduction_type: flatten
classification_head_1/dropout: 0.25
optimizer: adam
learning_rate: 0.001
Score: 0.07369301468133926
Trial summary
Hyperparameters:
res_net_block_1/pretrained: False
res_net_block_1/version: resnet50
res_net_block_1/imagenet_size: True
classification_head_1/spatial_reduction_1/reduction_type: flatten
classification_head_1/dropout: 0
optimizer: adam
learning_rate: 0.001
Score: 0.11127123236656189
Trial summary
Hyperparameters:
res_net_block_1/pretrained: False
res_net_block_1/version: resnet50
res_net_block_1/imagenet_size: True
classification_head_1/spatial_reduction_1/reduction_type: flatten
classification_head_1/dropout: 0
optimizer: adam
learning_rate: 0.1
Score: 0.274964302778244


### Display best model

In [6]:
best_resnet_model = resnet_auto_model.export_model()
best_resnet_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
cast_to_float32 (CastToFloat32) (None, 28, 28)       0           input_1[0][0]                    
__________________________________________________________________________________________________
expand_last_dim (ExpandLastDim) (None, 28, 28, 1)    0           cast_to_float32[0][0]            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 28, 28, 1)    3           expand_last_dim[0][0]            
_______________________________________________________________________________________

### Evaluate the best resnet model on the test data.

In [7]:

test_loss, test_acc = resnet_auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))



Accuracy: 75.14%


## XceptionBlock

In [ ]:
import autokeras as ak

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.XceptionBlock()(output_node)
output_node = ak.ClassificationHead()(output_node)

xception_auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=3,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
xception_auto_model.fit(x_train, y_train, epochs=10, batch_size=64)
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

### Display the best xception model

In [ ]:
best_xception_model = xception_auto_model.export_model()
tf.keras.utils.plot_model(best_xception_model, show_shapes=True, expand_nested=True) # rankdir='LR'

In [ ]:
best_xception_model.summary()

### Evaluate the best xception model on the test data.

In [ ]:

test_loss, test_acc = resnet_auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))



## HyperBlock for image classification (ImageBlock)

In [11]:
import autokeras as ak

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageBlock(
                # Normalize the dataset.
                normalize=True,
                # Do not do data augmentation.
                augment=False)(input_node)
output_node = ak.ClassificationHead()(output_node)

auto_model = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node,
    max_trials=3,
    overwrite=True,
    seed=42)

start_time = timeit.default_timer()
auto_model.fit(x_train, y_train, epochs=10, batch_size=64)
stop_time = timeit.default_timer()
print('Total time: {time} seconds.'.format(time=round(stop_time - start_time, 2)))

Trial 3 Complete [00h 33m 08s]
val_loss: 0.07790225744247437

Best val_loss So Far: 0.06216062977910042
Total elapsed time: 01h 39m 09s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
938/938 [==============================] - 228s 244ms/step - loss: 1.6475 - accuracy: 0.8704
Epoch 2/10
938/938 [==============================] - 229s 244ms/step - loss: 0.2597 - accuracy: 0.9593
Epoch 3/10
938/938 [==============================] - 228s 243ms/step - loss: 0.0841 - accuracy: 0.9783
Epoch 4/10
938/938 [==============================] - 228s 244ms/step - loss: 0.0590 - accuracy: 0.9831
Epoch 5/10
938/938 [==============================] - 228s 243ms/step - loss: 0.0742 - accuracy: 0.9829
Epoch 6/10
938/938 [==============================] - 228s 243ms/step - loss: 0.0450 - accuracy: 0.9865
Epoch 7/10
938/938 [==============================] - 228s 243ms/step - loss: 0.0352 - accuracy: 0.9894
Epoch 8/10
938/938 [==============================] - 228s 243ms/step - loss: 0.0296 - accuracy: 0

In [15]:
auto_model.tuner.results_summary()

Results summary
Results in ./auto_model
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
image_block_1/block_type: resnet
image_block_1/res_net_block_1/pretrained: False
image_block_1/res_net_block_1/version: resnet50
image_block_1/res_net_block_1/imagenet_size: True
classification_head_1/spatial_reduction_1/reduction_type: flatten
classification_head_1/dropout: 0
optimizer: adam
learning_rate: 0.001
Score: 0.06216062977910042
Trial summary
Hyperparameters:
image_block_1/block_type: resnet
image_block_1/res_net_block_1/pretrained: False
image_block_1/res_net_block_1/version: resnet50
image_block_1/res_net_block_1/imagenet_size: True
classification_head_1/spatial_reduction_1/reduction_type: flatten
classification_head_1/dropout: 0.25
optimizer: adam
learning_rate: 0.001
Score: 0.07790225744247437
Trial summary
Hyperparameters:
image_block_1/block_type: resnet
image_block_1/res_net_block_1/pretrained: False
image_block_1/res_net_block_1/ve

In [16]:
best_model = auto_model.export_model()
best_model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28)]     0                                            
__________________________________________________________________________________________________
cast_to_float32 (CastToFloat32) (None, 28, 28)       0           input_1[0][0]                    
__________________________________________________________________________________________________
expand_last_dim (ExpandLastDim) (None, 28, 28, 1)    0           cast_to_float32[0][0]            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 28, 28, 1)    3           expand_last_dim[0][0]            
_______________________________________________________________________________________

In [17]:

test_loss, test_acc = auto_model.evaluate(x_test, y_test, verbose=0)
print('Accuracy: {accuracy}%'.format(accuracy=round(test_acc * 100, 2)))



Accuracy: 98.91%


## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/preprocessor/#normalization-class),
[ImageAugmentation](/preprocessor/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/head/#classificationhead-class).